In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

CSV_PATH = os.path.join(os.getcwd(), 'results_3.2.csv')
PLOTS_DIR = os.path.join(os.getcwd(), 'plots')
os.makedirs(PLOTS_DIR, exist_ok=True)

res = pd.read_csv(CSV_PATH)
if 'user' not in res.columns:
    res['user'] = 'unknown'

res['n'] = res['n'].astype(int)
res['sparsity'] = res['sparsity'].astype(int)
res['reps'] = res['reps'].astype(int)
res['procs_label'] = res['procs'].astype(str)

def procs_to_int(x):
    if str(x).lower() == 'sequential':
        return -1
    try:
        return int(x)
    except Exception:
        return 1

res['procs'] = res['procs_label'].apply(procs_to_int)

# Component labels for timing breakdown
component_labels = {
    'time_csr_construct': 'CSR Construct',
    'time_send': 'MPI Send',
    'time_spmv': 'CSR SpMV',
    'time_dense_total': 'Dense'
}

In [2]:
# CSR (stacked) vs Dense comparison
def plot_csr_vs_dense_subplots(df, user):
    user_data = df[df['user'] == user].copy()
    if user_data.empty:
        return
    
    # Get unique n values and sparsity levels
    n_values = sorted(user_data['n'].unique())
    sparsity_values = sorted(user_data['sparsity'].unique())
    
    if len(n_values) == 0 or len(sparsity_values) == 0:
        return
    
    # Patterns for different n values
    n_patterns = {}
    patterns_list = ['', '..', '//', 'xx', '\\\\', '||', '--', '++', 'oo', '**']
    for i, n in enumerate(n_values):
        n_patterns[n] = patterns_list[i % len(patterns_list)]
    
    # Create subplots (2x2 grid) - one for each sparsity
    nplots = min(len(sparsity_values), 4)
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    for idx in range(nplots):
        sparsity = sparsity_values[idx]
        
        data = user_data[user_data['sparsity'] == sparsity].copy()
        if data.empty:
            continue
        
        ax = axes[idx]
        
        # Get unique process counts
        procs_values = sorted(data['procs'].unique())
        x_positions = np.arange(len(procs_values))
        
        # Calculate bar width based on number of n values
        num_bars = len(n_values) * 2  # CSR and Dense for each n
        bar_width = 0.8 / num_bars if num_bars > 0 else 0.8
        
        bar_idx = 0
        for n in n_values:
            n_data = data[data['n'] == n].copy()
            if n_data.empty:
                bar_idx += 2
                continue
            
            summary_mean = n_data.groupby('procs').mean(numeric_only=True).reset_index()
            summary_std = n_data.groupby('procs').std(numeric_only=True).reset_index().fillna(0)
            summary_mean = summary_mean.sort_values('procs')
            summary_std = summary_std.set_index('procs').reindex(summary_mean['procs']).reset_index(drop=True)
            
            # Ensure we have data for all process values
            procs_data = summary_mean['procs'].values
            if len(procs_data) == 0:
                bar_idx += 2
                continue
            
            # Get CSR components (including time_send for MPI overhead)
            csr_construct_mean = summary_mean['time_csr_construct'].values
            csr_send_mean = summary_mean['time_send'].values
            csr_spmv_mean = summary_mean['time_spmv'].values
            csr_construct_std = summary_std['time_csr_construct'].values
            csr_send_std = summary_std['time_send'].values
            csr_spmv_std = summary_std['time_spmv'].values
            
            # Calculate total CSR error (error propagation for sum of 3 components)
            csr_total_mean = csr_construct_mean + csr_send_mean + csr_spmv_mean
            csr_total_std = np.sqrt(csr_construct_std**2 + csr_send_std**2 + csr_spmv_std**2)
            csr_total_std = np.minimum(csr_total_std, csr_total_mean)
            
            # Get Dense
            dense_mean = summary_mean['time_dense_total'].values
            dense_std = np.minimum(summary_std['time_dense_total'].values, dense_mean)
            
            # Calculate offsets for this n
            offset_csr = bar_idx * bar_width - 0.4 + bar_width/2
            offset_dense = (bar_idx + 1) * bar_width - 0.4 + bar_width/2
            
            # Map processes to x positions
            x_pos = []
            for p in procs_data:
                if p in procs_values:
                    x_pos.append(procs_values.index(p))
            x_pos = np.array(x_pos)
            
            # CSR bar (stacked) - bottom without error bar
            ax.bar(x_pos + offset_csr, csr_construct_mean, bar_width, 
                   label=f'CSR Construct (n={n})' if bar_idx == 0 else None,
                   color='#4C72B0', alpha=0.85, edgecolor='black', linewidth=0.5,
                   hatch=n_patterns[n])
            
            # CSR bar (stacked) - middle layer (MPI Send)
            ax.bar(x_pos + offset_csr, csr_send_mean, bar_width, 
                   bottom=csr_construct_mean,
                   label=f'MPI Send (n={n})' if bar_idx == 0 else None,
                   color='#DD8452', alpha=0.85, edgecolor='black', linewidth=0.5,
                   hatch=n_patterns[n])
            
            # CSR bar (stacked) - top with total error bar
            ax.bar(x_pos + offset_csr, csr_spmv_mean, bar_width, 
                   bottom=csr_construct_mean + csr_send_mean,
                   yerr=csr_total_std, capsize=3,
                   label=f'CSR SpMV (n={n})' if bar_idx == 0 else None,
                   color='#55A868', alpha=0.85, edgecolor='black', linewidth=0.5,
                   hatch=n_patterns[n])
            
            # Dense bar
            ax.bar(x_pos + offset_dense, dense_mean, bar_width, 
                   yerr=dense_std, capsize=3,
                   label=f'Dense (n={n})' if bar_idx == 0 else None,
                   color='#C44E52', alpha=0.85, edgecolor='black', linewidth=0.5,
                   hatch=n_patterns[n])
            
            bar_idx += 2
        
        ax.set_xticks(x_positions)
        ax.set_xticklabels([str(int(p)) if p != -1 else 'seq' for p in procs_values])
        ax.set_xlabel('Processes')
        ax.set_ylabel('Time (s)')
        ax.set_yscale('log')
        ax.set_title(f'Sparsity = {sparsity}%')
        ax.grid(axis='y', alpha=0.25)
        if idx == 0:
            # Create legend with colors and patterns
            from matplotlib.patches import Patch
            handles = []
            labels = []
            
            # First add color legend (components)
            handles.append(Patch(facecolor='#4C72B0', edgecolor='black', alpha=0.85))
            labels.append('CSR Construct')
            handles.append(Patch(facecolor='#DD8452', edgecolor='black', alpha=0.85))
            labels.append('MPI Send')
            handles.append(Patch(facecolor='#55A868', edgecolor='black', alpha=0.85))
            labels.append('CSR SpMV')
            handles.append(Patch(facecolor='#C44E52', edgecolor='black', alpha=0.85))
            labels.append('Dense')
            
            # Add separator
            handles.append(Patch(facecolor='white', edgecolor='white'))
            labels.append('')
            
            # Then add pattern legend (n values)
            for n in n_values:
                handles.append(Patch(facecolor='lightgray', edgecolor='black', hatch=n_patterns[n]))
                labels.append(f'n={n}')
            
            ax.legend(handles, labels, fontsize=8, loc='upper right', ncol=1)
    
    # Hide unused subplots
    for idx in range(nplots, 4):
        axes[idx].axis('off')
    
    fig.suptitle(f'CSR Total vs Dense for Different Matrix Sizes (User: {user})', fontsize=14)
    plt.tight_layout()
    
    fname = os.path.join(PLOTS_DIR, f'csr_vs_dense_all_{user}.png')
    fig.savefig(fname, dpi=300)
    plt.close(fig)

In [ ]:
# CSR parallel speedup (baseline = sequential CSR) 
def plot_csr_parallel_speedup_subplots(df, user):
    user_data = df[df['user'] == user].copy()
    if user_data.empty:
        return

    # Get unique n values
    n_values = sorted(user_data['n'].unique())
    if len(n_values) == 0:
        return

    # Create subplots (2x2 grid)
    nplots = min(len(n_values), 4)
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()

    for idx, n in enumerate(n_values[:nplots]):
        data = user_data[user_data['n'] == n].copy()
        if data.empty:
            continue

        grouped = data.groupby(['sparsity','procs']).mean(numeric_only=True).reset_index()
        ax = axes[idx]

        sparsities = sorted(grouped['sparsity'].unique())
        # Create offset for each sparsity level
        offset_step = 0.15
        offsets = np.linspace(-offset_step * (len(sparsities)-1)/2,
                              offset_step * (len(sparsities)-1)/2,
                              len(sparsities))

        for sparsity_idx, (s, sdata) in enumerate(grouped.groupby('sparsity')):
            sdata = sdata.sort_values('procs')
            # Use raw data for baseline detection (sequential CSR Total = Construct + Send + SpMV)
            raw_s = data[data['sparsity'] == s]
            baseline_raw = raw_s[raw_s['procs_label'].str.lower() == 'sequential']
            if not baseline_raw.empty:
                b_construct = baseline_raw['time_csr_construct'].mean()
                b_send = baseline_raw['time_send'].mean()
                b_spmv = baseline_raw['time_spmv'].mean()
                b = b_construct + b_send + b_spmv  # CSR Total
                sb_construct = baseline_raw['time_csr_construct'].std() if baseline_raw['time_csr_construct'].std() else 0.0
                sb_send = baseline_raw['time_send'].std() if baseline_raw['time_send'].std() else 0.0
                sb_spmv = baseline_raw['time_spmv'].std() if baseline_raw['time_spmv'].std() else 0.0
                sb = np.sqrt(sb_construct**2 + sb_send**2 + sb_spmv**2)  # Error propagation for sum
            else:
                raw_1 = raw_s[raw_s['procs'] == 1]
                if raw_1.empty:
                    continue
                b_construct = raw_1['time_csr_construct'].mean()
                b_send = raw_1['time_send'].mean()
                b_spmv = raw_1['time_spmv'].mean()
                b = b_construct + b_send + b_spmv
                sb_construct = raw_1['time_csr_construct'].std() if not np.isnan(raw_1['time_csr_construct'].std()) else 0.0
                sb_send = raw_1['time_send'].std() if not np.isnan(raw_1['time_send'].std()) else 0.0
                sb_spmv = raw_1['time_spmv'].std() if not np.isnan(raw_1['time_spmv'].std()) else 0.0
                sb = np.sqrt(sb_construct**2 + sb_send**2 + sb_spmv**2)

            # Compute CSR Total means and stds for each process (excluding sequential)
            raw_parallel = raw_s[raw_s['procs'] != -1]
            csr_stats = raw_parallel.groupby('procs').agg({
                'time_csr_construct': ['mean', 'std'],
                'time_send': ['mean', 'std'],
                'time_spmv': ['mean', 'std']
            }).reset_index()
            csr_stats.columns = ['procs', 'construct_mean', 'construct_std', 'send_mean', 'send_std', 'spmv_mean', 'spmv_std']
            csr_stats['csr_total_mean'] = csr_stats['construct_mean'] + csr_stats['send_mean'] + csr_stats['spmv_mean']
            csr_stats['csr_total_std'] = np.sqrt(csr_stats['construct_std'].fillna(0)**2 + 
                                                   csr_stats['send_std'].fillna(0)**2 +
                                                   csr_stats['spmv_std'].fillna(0)**2)
            csr_stats = csr_stats.sort_values('procs').reset_index(drop=True)

            if csr_stats.empty:
                continue

            m = csr_stats['csr_total_mean'].values
            sm = csr_stats['csr_total_std'].values
            speedup_mean = b / m
            # Propagate uncertainty: var = (sb^2)/(m^2) + (b^2)*(sm^2)/(m^4)
            var = (sb**2) / (m**2) + (b**2) * (sm**2) / (m**4)
            speedup_std = np.sqrt(var)
            speedup_std = np.maximum(speedup_std, 0)

            # Apply offset to x-axis positions
            x_pos = csr_stats['procs'].values + offsets[sparsity_idx]
            ax.errorbar(x_pos, speedup_mean, yerr=speedup_std,
                       marker='o', label=f's={s}%', capsize=4)

        # Add baseline reference at 1.0
        ax.axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='No speedup (1.0x)')
        
        # Show process counts on the x-axis (excluding sequential)
        all_procs = sorted([p for p in grouped['procs'].unique() if p != -1])
        ax.set_xticks(all_procs)
        ax.set_xticklabels([str(int(p)) for p in all_procs])
        ax.set_xlabel('Processes')
        ax.set_ylabel('Speedup (x)')
        ax.set_title(f'n={n}')
        ax.grid(True, alpha=0.25)
        ax.legend(fontsize=8)

    # Hide unused subplots
    for idx in range(nplots, 4):
        axes[idx].axis('off')

    fig.suptitle(f'CSR Total Parallel Speedup vs Process Count (User: {user})', fontsize=14)
    plt.tight_layout()

    fname = os.path.join(PLOTS_DIR, f'csr_parallel_speedup_all_{user}.png')
    fig.savefig(fname, dpi=300)
    plt.close(fig)


# Dense vs CSR speedup (dense / csr) with same formatting
def plot_dense_vs_csr_subplots(df, user):
    user_data = df[df['user'] == user].copy()
    if user_data.empty:
        return

    # Get unique n values
    n_values = sorted(user_data['n'].unique())
    if len(n_values) == 0:
        return

    # Create subplots (2x2 grid)
    nplots = min(len(n_values), 4)
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()

    for idx, n in enumerate(n_values[:nplots]):
        data = user_data[user_data['n'] == n].copy()
        if data.empty:
            continue

        grouped = data.groupby(['sparsity','procs']).mean(numeric_only=True).reset_index()
        ax = axes[idx]

        sparsities = sorted(grouped['sparsity'].unique())
        # Create offset for each sparsity level
        offset_step = 0.15
        offsets = np.linspace(-offset_step * (len(sparsities)-1)/2,
                              offset_step * (len(sparsities)-1)/2,
                              len(sparsities))

        for sparsity_idx, (s, sdata) in enumerate(grouped.groupby('sparsity')):
            sdata = sdata.sort_values('procs')
            
            # Filter out sequential data - we compare parallel to parallel
            sdata_parallel = sdata[sdata['procs'] != -1]
            if sdata_parallel.empty:
                continue
            
            # Use raw data to compute per-process means and stds
            raw_s = data[data['sparsity'] == s]
            raw_parallel = raw_s[raw_s['procs'] != -1]
            
            # Compute CSR Total (Construct + Send + SpMV) for each process
            csr_stats = raw_parallel.groupby('procs').agg({
                'time_csr_construct': ['mean', 'std'],
                'time_send': ['mean', 'std'],
                'time_spmv': ['mean', 'std']
            }).reset_index()
            
            csr_stats.columns = ['procs', 'construct_mean', 'construct_std', 'send_mean', 'send_std', 'spmv_mean', 'spmv_std']
            csr_stats['csr_total_mean'] = csr_stats['construct_mean'] + csr_stats['send_mean'] + csr_stats['spmv_mean']
            csr_stats['csr_total_std'] = np.sqrt(csr_stats['construct_std'].fillna(0)**2 + 
                                                   csr_stats['send_std'].fillna(0)**2 +
                                                   csr_stats['spmv_std'].fillna(0)**2)
            
            # Compute Dense stats for each process
            dense_stats = raw_parallel.groupby('procs')['time_dense_total'].agg(['mean', 'std']).reset_index()
            dense_stats.columns = ['procs', 'dense_mean', 'dense_std']
            dense_stats['dense_std'] = dense_stats['dense_std'].fillna(0)
            
            # Merge CSR and Dense stats
            merged = csr_stats.merge(dense_stats, on='procs')
            if merged.empty:
                continue
            
            # Calculate speedup: Dense / CSR (both parallel with same process count)
            b = merged['csr_total_mean'].values
            sb = merged['csr_total_std'].values
            m = merged['dense_mean'].values
            sm = merged['dense_std'].values
            
            speedup_mean = m / b
            # Propagate uncertainty: var = (sm^2)/(b^2) + (m^2)*(sb^2)/(b^4)
            var = (sm**2) / (b**2) + (m**2) * (sb**2) / (b**4)
            speedup_std = np.sqrt(var)
            speedup_std = np.maximum(speedup_std, 0)

            x_pos = merged['procs'].values + offsets[sparsity_idx]
            ax.errorbar(x_pos, speedup_mean, yerr=speedup_std,
                       marker='o', label=f's={s}%', capsize=4)

        # Add baseline reference at 1.0
        ax.axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Equal performance (1.0x)')
        
        # Show process counts on the x-axis (excluding sequential)
        all_procs = sorted([p for p in grouped['procs'].unique() if p != -1])
        ax.set_xticks(all_procs)
        ax.set_xticklabels([str(int(p)) for p in all_procs])
        ax.set_xlabel('Processes')
        ax.set_ylabel('Speedup (Dense / CSR)')
        ax.set_title(f'n={n}')
        ax.grid(True, alpha=0.25)
        ax.legend(fontsize=8)

    # Hide unused subplots
    for idx in range(nplots, 4):
        axes[idx].axis('off')

    fig.suptitle(f'Dense vs CSR Performance Ratio (User: {user})', fontsize=14)
    plt.tight_layout()

    fname = os.path.join(PLOTS_DIR, f'dense_vs_csr_speedup_all_{user}.png')
    fig.savefig(fname, dpi=300)
    plt.close(fig)

In [4]:
# Generate plots per user
for user in sorted(res['user'].unique()):
    user_df = res[res['user'] == user]
    print(f'Generating plots for user: {user}')

    plot_csr_vs_dense_subplots(res, user)
    plot_csr_parallel_speedup_subplots(res, user)
    plot_dense_vs_csr_subplots(res, user)

print('All plots generated and saved to', PLOTS_DIR)


Generating plots for user: ea24205
Generating plots for user: marr
Generating plots for user: phoebus
All plots generated and saved to /home/marr/threads/Thread-Experiments/3_2_sparse_array_vector_multiplication/plots
